In [1]:
# Saving the Best Model...

#!pip install mlxtend
#!pip3 install numpy==1.24.0

In [2]:
# Forward Feature Selection method...


import pandas as pd
import numpy as np

#print(np.__version__)

import pickle
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from mlxtend.feature_selection import SequentialFeatureSelector

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE


In [3]:
def forward_selection(indep, dep,n):
    logistic_model=LogisticRegression(solver='saga', max_iter=10)
    
    bs = SequentialFeatureSelector(estimator=logistic_model, k_features=n, forward=True,floating=False,
                                    verbose=0,scoring=None,cv=5,n_jobs=1)
    
    fit1 = bs.fit(indep, dep)
    x_new= fit1.transform(indep)

    # get the selected feature names...
    selected_features=fit1.k_feature_names_
    
    return x_new, selected_features


def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test,scX


def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    

# Displaying the values in the tabular frame

def FW_selection(accuracy_score):
    
    df=pd.DataFrame(index=["Logistic"],columns=["Logistic"])
    df["Logistic"]=accuracy_score
    
    return df




In [4]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]
  
    
x_new, selected_features =forward_selection(indep, dep,4)

x_new

/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.

array([[3.        , 0.        , 3.07735602, 4.70559701],
       [2.        , 0.        , 0.7       , 4.70559701],
       [1.        , 0.        , 0.6       , 4.70559701],
       ...,
       [3.        , 0.        , 6.        , 3.4       ],
       [0.        , 0.        , 6.8       , 4.70559701],
       [0.        , 0.        , 1.        , 4.9       ]])

In [5]:
x_train,x_test,y_train,y_test,scX = Split_Scaler(x_new, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)

result= FW_selection(accuracy_score)

In [6]:
print("Accuracy: ",accuracy_score)

Accuracy:  0.9416666666666667


In [7]:
result #4

,Logistic
Logistic,0.941667


In [8]:
# Save the Model
pickle.dump(classifier,open("finalized_logmodel_mlxtend02",'wb'))

# save the Standard scalar object
pickle.dump(scX,open("scX_002.pkl",'wb'))

In [9]:
print(selected_features)


('al', 'su', 'sc', 'rc')


In [10]:
# mlxtend returned the result as Tuples. So, you can convert them to strings using the following approach:

selected_features = [''.join(feature)  for feature in selected_features]
print(selected_features)

['al', 'su', 'sc', 'rc']


In [11]:
df1[selected_features]

,al,su,sc,rc
0,3.0,0.0,3.077356,4.705597
1,2.0,0.0,0.700000,4.705597
2,1.0,0.0,0.600000,4.705597
3,1.0,0.0,0.700000,4.705597
4,0.0,0.0,0.600000,4.705597
...,...,...,...,...
394,0.0,0.0,1.300000,4.400000
395,0.0,2.0,2.800000,4.705597
396,3.0,0.0,6.000000,3.400000
397,0.0,0.0,6.800000,4.705597
